# Initialization

In [0]:
from statistics import mean
from timeit import default_timer as timer
from math import sqrt, exp
from random import shuffle, random, randint
from google.colab import files
import copy
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [0]:
# Download the "Coors_140.csv" from the GitHub repository and upload it
uploaded = files.upload()

Saving Coors_140.csv to Coors_140.csv


In [0]:
coors = pd.read_csv("Coors_140.csv") 
coords = coors.to_numpy()
coords = np.delete(coords, 0,1)
amount_of_coordinates = 40
Coordinates = coords[:amount_of_coordinates]

#Plotting
plt.scatter(Coordinates[0][1], Coordinates[0][2], c='r', marker='D')
plt.scatter(Coordinates[1:,1], Coordinates[1:,2])

In [0]:
# Or else, if you prefer to generate the coordinates randomly, follow this step.

nodes = 40
Coordinates = np.ones((nodes, 3))
Coordinates[0] = 1, 0.5, 0.5

for i in range(1, nodes):
  Coordinates[i] = int(i+1), random(), random()

#Plotting
plt.scatter(Coordinates[0][1], Coordinates[0][2], c='r', marker='D')
plt.scatter(Coordinates[1:,1], Coordinates[1:,2])

# Distance Matrix

In [0]:
def Distance(x1, y1, x2, y2):
    return sqrt((x1-x2)**2+(y1-y2)**2)

#Creating Distance Matrix
length = len(Coordinates)
Distance_Matrix = np.zeros((length, length))
for i in range(length):
  for j in range(length):
    Distance_Matrix[i][j] = Distance(Coordinates[i][1],  Coordinates[i][2], Coordinates[j][1], Coordinates[j][2])

# Functions


In [0]:
def First_Tour():
  FirstTour = list(range(2, length+1))
  shuffle(FirstTour)
  FirstTour = [1] + FirstTour + [1]
  return FirstTour

def NewRandom(tour):
  tmp=tour[0]
  del tour[0], tour[-1]
  
  shuffle(tour)
  tour = [tmp] + tour + [tmp]
  return tour

def Plotting(tour, coords):
    n=len(tour)
    longitude=[]
    latitude=[]
    
    for i in range(n):
            longitude.append(coords[tour[i]-1, 1])
            latitude.append(coords[tour[i]-1, 2])
    plt.figure()    
    plt.plot([longitude[i] for i in range(n)],
              [latitude[i] for i in range(n)],'-.', color='cornflowerblue')
    plt.scatter(Coordinates[0][1], Coordinates[0][2], c = 'r', marker='D')
    plt.scatter(Coordinates[1:,1], Coordinates[1:,2])

def FindCurrentCost(CurTour):
  temporaryDis=0
  for i in range(len(CurTour)-1):    
    temporaryDis += Distance_Matrix[CurTour[i]-1,CurTour[i+1]-1]
  return temporaryDis 

def FirstParents(n,startingtour):
  lst=[]    
  for i in range(n):
    tmp=startingtour[0]
    del startingtour[0], startingtour[-1]
    shuffle(startingtour)
    startingtour = [tmp] + startingtour + [tmp]
    lst.append(copy.copy(startingtour))
  return lst

def CostofList(lis):
  costs=[]
  for i in range(len(lis)):
    a=FindCurrentCost(lis[i])      
    costs.append(a)
  return costs
  
def pickone(roullete):
    num=random()
    for i in range(Population):
        if num >= roullete[i,0] and num <= roullete[i,1]:
            idx = i
            break
    return idx  

# Selection


In [0]:
def Elitism(lst, lstcost, percentage):
  elite = []
  num = (len(lst)*(percentage))//100

  for i in range(num):
    idx=lstcost.index(min(lstcost))
    elite.append(lst[idx])
    lstcost[idx]=99999999

  return elite

def RoulleteWheelSelectionArray(costs):
    n = len(costs)
    minim=min(costs)
    maxim=max(costs)
    reverse=np.zeros(n)
    normal=np.zeros(n)
    prop_sel=np.zeros((n,3))
    
    for i in range(n):
        reverse[i]=(maxim-costs[i])+minim
    summ=sum(reverse) 
    prop_sel[0,0]=0      
    for i in range(n-1):
        normal[i]= reverse[i]/summ
        prop_sel[i,1] = normal[i] + prop_sel[i,0]
        prop_sel[i,2] = normal[i]
        prop_sel[i+1,0] = prop_sel[i,1]
       
    normal[n-1]= reverse[n-1]/summ    
    prop_sel[n-1,1] = normal[n-1] + prop_sel[n-1,0]    
    prop_sel[n-1,2] = normal[n-1] 
    
    return prop_sel

# Mutation


In [0]:
def Mut_Inversion(parent):
    tmp=parent[0]
    del parent[0],parent[-1]      

    i = randint(0,  len(parent)- 2)
    j = randint(i+2, len(parent))

    parent[i:j] = reversed(parent[i:j])
    
    parent.insert(0,tmp)
    parent.append(tmp)
    
    return parent

# Crossover


In [0]:
def Crossover_PMX(par1,par2):
  del par1[0],par2[0],par1[-1],par2[-1]
  i = randint(0,  len(par1)- 3)
  j = randint(i+3, len(par1))

  tmp1 = par1[i:j]
  tmp2 = par2[i:j]

  off2 = [None] * len(par1)
  off1 = [None] * len(par1)

  off1[i:j]=par2[i:j]
  off2[i:j]=par1[i:j]  

  for k in range(len(par1)):
    if off2[k] == None:
      if  not par2[k] in off2:
        off2[k] = par2[k]
      else:
        for x in tmp2:
          if not x in off2:
            off2[k] = x
          else:continue
    else:continue
  off2 = [1] + off2 + [1]

  for k in range(len(par1)):
    if off1[k] == None:
      if  not par1[k] in off1:
        off1[k] = par1[k]
      else:
        for x in tmp1:
          if not x in off1:
            off1[k] = x
          else:continue
    else:continue
  off1 = [1] + off1 + [1]

  return off1,off2

# Genetic Algorithm

In [0]:
Population = 150
Generations = 800
FirstTour=First_Tour()
Parents=FirstParents(Population,FirstTour[:])
Parents_Cost_List=CostofList(Parents[:])
elitism = 5 # integer percentage of 100

# Propabilitties
Crossover_Propabillity = 75
Mutation_Propabillity = 24
New_Random_Propabillity = 100 - Crossover_Propabillity - Mutation_Propabillity

Shortest_Tour_Graph = []

start = timer()
for rnd in range(Generations):
  Proporsional_Selection_Array = RoulleteWheelSelectionArray(Parents_Cost_List)
  Children = Elitism(Parents, Parents_Cost_List, elitism)
  while True:
    randomnum=randint(1,100)

    if randomnum <= New_Random_Propabillity:
      Children.append(NewRandom(FirstTour[:]))
      
    elif randomnum > New_Random_Propabillity and randomnum <= (Mutation_Propabillity + New_Random_Propabillity) :
      idx=pickone(Proporsional_Selection_Array)
      chromosome = Parents[idx]
      Children.append(Mut_Inversion(chromosome[:]))

    elif randomnum > (Mutation_Propabillity + New_Random_Propabillity):
      idx1=pickone(Proporsional_Selection_Array)
      while True:
        idx2=pickone(Proporsional_Selection_Array)
        if idx2 != idx1:break
      kid1, kid2 = Crossover_PMX(Parents[idx1][:], Parents[idx2][:])
      Children.append(kid1)
      if len(Children) == Population:break
      Children.append(kid2)

    if len(Children) == Population:break

  Children_Cost_List = CostofList(Children)
  
  # Children -> Parents
  Parents = Children[:]
  Parents_Cost_List = Children_Cost_List[:]

  # Ploting
  Shortest_Tour_Graph.append(min(Children_Cost_List)) 

Shortest_Tour = Parents[best]
Shortest_Tour_Cost = FindCurrentCost(Shortest_Tour)
end = timer() 

print("Shortest Tour:",Shortest_Tour)
print("Shortest Tour cost:","{0:.3f}".format(round(Shortest_Tour_Cost,3)))
print("Calculation Time:","{0:.4f}".format(round((end - start),4)), "sec")
Plotting(Shortest_Tour, Coordinates)
plt.figure()
plt.plot(Shortest_Tour_Graph,'r')